# TinyCNN Training with Google Drive

這個 Notebook 用於：
1. 掛載 Google Drive
2. 從 Drive 中載入資料
3. 定義並訓練 TinyCNN 模型
4. 保存最佳模型

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
img_size      = 48
batch_size    = 64
num_epochs    = 20
learning_rate = 1e-3
save_path     = '/content/drive/MyDrive/archive/cnn_small.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [ ]:
train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
test_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

train_ds = datasets.ImageFolder('/content/drive/MyDrive/archive/train', transform=train_tf)
test_ds  = datasets.ImageFolder('/content/drive/MyDrive/archive/test',   transform=test_tf)
train_loader = DataLoader(train_ds, batch_size, shuffle=True,  num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size, shuffle=False, num_workers=2)

print('Classes:', train_ds.classes)
print('Num classes:', len(train_ds.classes))

Classes: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Num classes: 7


In [ ]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.pool  = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4,4))
        self.dropout = nn.Dropout(0.4)
        self.fc1 = nn.Linear(64 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model     = TinyCNN(num_classes=len(train_ds.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
best_acc = 0.0
for epoch in range(1, num_epochs+1):
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)
    acc = correct / total * 100
    print(f"Epoch {epoch:02d} — Val Acc: {acc:.2f}% (best {best_acc:.2f}%)")
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), save_path)
        print(f" New best model saved to {save_path}\n")

print(f"Training finished. Best accuracy: {best_acc:.2f}%")

Epoch 01 — Val Acc: 42.31% (best 0.00%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 02 — Val Acc: 44.79% (best 42.31%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 03 — Val Acc: 45.99% (best 44.79%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 04 — Val Acc: 47.30% (best 45.99%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 05 — Val Acc: 49.33% (best 47.30%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 06 — Val Acc: 50.28% (best 49.33%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 07 — Val Acc: 50.26% (best 50.28%)
Epoch 08 — Val Acc: 51.20% (best 50.28%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 09 — Val Acc: 51.62% (best 51.20%)
 New best model saved to /content/drive/MyDrive/archive/cnn_small.pth

Epoch 10 — Val Acc: 52.88% (best 51.62%)
 New best model saved t